In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB or IS_KAGGLE:
    !apt update && apt install -y libpq-dev libsdl2-dev swig xorg-dev xvfb
    %pip install -U tf-agents pyvirtualdisplay
    %pip install -U gym>=0.21.0
    %pip install -U gym[box2d,atari,accept-rom-license]

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rl"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 

This was inspired by using a combination of labs, ray website, https://colab.research.google.com/github/ageron/handson-ml2/blob/master/18_reinforcement_learning.ipynb and https://www.anyscale.com/blog/an-introduction-to-reinforcement-learning-with-openai-gym-rllib-and-google

In [4]:
pip install ray==1.12.0


     |████████████████████████████████| 53.2 MB 1.4 MB/s 
     |████████████████████████████████| 4.1 MB 34.8 MB/s 
     |████████████████████████████████| 144 kB 49.0 MB/s 
     |████████████████████████████████| 8.8 MB 36.7 MB/s 
     |████████████████████████████████| 461 kB 47.8 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [5]:
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total

In [6]:
pip install gym==0.22

     |████████████████████████████████| 631 kB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.22.0-py3-none-any.whl size=708393 sha256=61e904c912c8ed730b61a47f6e775c7259094ab5a57cf6ae3e7a1dba1ea71331
  Stored in directory: /root/.cache/pip/wheels/7d/5e/87/7d50e0179edda70feff5bba05c381041e1c1fd80c6b06a4cc3
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.23.1
    Uninstalling gym-0.23.1:
      Successfully uninstalled gym-0.23.1


In [7]:
pip install -U tensorboardx

     |████████████████████████████████| 125 kB 5.3 MB/s 


In [8]:
pip install lz4

     |████████████████████████████████| 1.2 MB 5.6 MB/s 


In [9]:
import gym
env = gym.make("CartPole-v0")


/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:506: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  f"The environment {path} is out of date. You should consider "


In [10]:
env.reset()

for i in range(20):

  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())

  print("step", i, observation, reward, done, info)

env.close()

step 0 [ 0.04664953 -0.15030913 -0.0198714   0.3197851 ] 1.0 False {}
step 1 [ 0.04364334 -0.3451425  -0.0134757   0.60613567] 1.0 False {}
step 2 [ 0.03674049 -0.14983477 -0.00135299  0.30923888] 1.0 False {}
step 3 [ 0.0337438  -0.3449374   0.00483179  0.6014948 ] 1.0 False {}
step 4 [ 0.02684505 -0.14988337  0.01686169  0.31033772] 1.0 False {}
step 5 [ 0.02384738 -0.34524146  0.02306844  0.60829026] 1.0 False {}
step 6 [ 0.01694255 -0.5406782   0.03523425  0.9081489 ] 1.0 False {}
step 7 [ 0.00612899 -0.7362589   0.05339722  1.2116946 ] 1.0 False {}
step 8 [-0.00859619 -0.5418654   0.07763112  0.9362113 ] 1.0 False {}
step 9 [-0.0194335  -0.7379436   0.09635534  1.2522434 ] 1.0 False {}
step 10 [-0.03419237 -0.544179    0.12140021  0.9912294 ] 1.0 False {}
step 11 [-0.04507595 -0.74069804  0.1412248   1.3194424 ] 1.0 False {}
step 12 [-0.05988991 -0.54761547  0.16761364  1.0740842 ] 1.0 False {}
step 13 [-0.07084222 -0.74450773  0.18909533  1.4143316 ] 1.0 False {}
step 14 [-0.0857

/usr/local/lib/python3.7/dist-packages/gym/envs/classic_control/cartpole.py:161: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  "You are calling 'step()' even though this "


In [11]:
# install dependencies needed for recording videos
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  x11-utils
0 upgraded, 1 newly installed, 0 to remove and 56 not upgraded.
Need to get 196 kB of archives.
After this operation, 650 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Fetched 196 kB in 1s (276 kB/s)
Selecting previously unselected package x11-utils.
(Reading database ... 157592 files and directories currently installed.)
Preparing to unpack .../x11-utils_7.7+3build1_amd64.deb ...
Unpacking x11-utils (7.7+3build1) ...
Setting up x11-utils (7.7+3build1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Attempting uninstall: pyvirtualdisplay
    Found existing installation: PyVirtualDisplay 3.0
    Uninstalling PyVirtualDisplay-3.0:
      Successfu

In [12]:
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
_ = display.start()

In [13]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(200):
  video.capture_frame()
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())
  # Not printing this time
  #print("step", i, observation, reward, done, info)

video.close()
env.close()

/usr/local/lib/python3.7/dist-packages/gym/envs/classic_control/cartpole.py:161: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  "You are calling 'step()' even though this "


In [14]:
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'


In [15]:

from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

In [16]:
import ray
from ray.rllib.agents.ppo import PPOTrainer
config = {
    "env": "CartPole-v0",
    # Change the following line to `“framework”: “tf”` to use tensorflow
    "framework": "torch",
    "model": {
      "fcnet_hiddens": [32],
      "fcnet_activation": "linear",
    },
}
stop = {"episode_reward_mean": 195}
ray.shutdown()
ray.init(
  num_cpus=3,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)
# execute training 
analysis = ray.tune.run(
  "PPO",
  config=config,
  stop=stop,
  checkpoint_at_end=True,
)

2022-04-24 09:11:19,748	INFO trial_runner.py:803 -- starting PPO_CartPole-v0_80d83_00000


Trial name,status,loc
PPO_CartPole-v0_80d83_00000,RUNNING,172.28.0.2:3156


Trial name,status,loc
PPO_CartPole-v0_80d83_00000,RUNNING,172.28.0.2:3156


Result for PPO_CartPole-v0_80d83_00000:
  agent_timesteps_total: 4000
  custom_metrics: {}
  date: 2022-04-24_09-11-44
  done: false
  episode_len_mean: 23.156976744186046
  episode_media: {}
  episode_reward_max: 69.0
  episode_reward_mean: 23.156976744186046
  episode_reward_min: 9.0
  episodes_this_iter: 172
  episodes_total: 172
  experiment_id: 495e8b46f96c44a5bba45679f3ae141f
  hostname: 4671fbd62325
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.20000000000000004
          cur_lr: 5.0000000000000016e-05
          entropy: 0.6839196100029894
          entropy_coeff: 0.0
          kl: 0.009778247770419302
          policy_loss: -0.020816732550500542
          total_loss: 9.190495002910655
          vf_explained_var: -0.0009381812105896652
          vf_loss: 9.209356138783116
        model: {}
        num_agent_steps_trained: 128.0
    num_agent_steps_sampled: 4000
    num_agent

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v0_80d83_00000,RUNNING,172.28.0.2:3156,1,8.63111,4000,23.157,69,9,23.157


...

Result for PPO_CartPole-v0_80d83_00000:
  agent_timesteps_total: 340000
  custom_metrics: {}
  date: 2022-04-24_09-23-40
  done: false
  episode_len_mean: 193.55
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 193.55
  episode_reward_min: 111.0
  episodes_this_iter: 21
  episodes_total: 2282
  experiment_id: 495e8b46f96c44a5bba45679f3ae141f
  hostname: 4671fbd62325
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 1.6543612251060557e-25
          cur_lr: 5.0000000000000016e-05
          entropy: 0.5836896336847736
          entropy_coeff: 0.0
          kl: 0.00019154569277586418
          policy_loss: -0.005443457808465727
          total_loss: 5.535590509958165
          vf_explained_var: -0.11281915326272288
          vf_loss: 5.541033950800537
        model: {}
        num_agent_steps_trained: 128.0
    num_agent_steps_sampled: 340000
    num_agent_steps_trained

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v0_80d83_00000,RUNNING,172.28.0.2:3156,85,721.924,340000,193.55,200,111,193.55


Result for PPO_CartPole-v0_80d83_00000:
  agent_timesteps_total: 344000
  custom_metrics: {}
  date: 2022-04-24_09-23-49
  done: true
  episode_len_mean: 196.11
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 196.11
  episode_reward_min: 111.0
  episodes_this_iter: 20
  episodes_total: 2302
  experiment_id: 495e8b46f96c44a5bba45679f3ae141f
  hostname: 4671fbd62325
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 8.271806125530278e-26
          cur_lr: 5.0000000000000016e-05
          entropy: 0.5750028940939134
          entropy_coeff: 0.0
          kl: 0.0005383986887342405
          policy_loss: 0.004769039234166504
          total_loss: 6.518303403803097
          vf_explained_var: -0.1406329942005937
          vf_loss: 6.513534375929063
        model: {}
        num_agent_steps_trained: 128.0
    num_agent_steps_sampled: 344000
    num_agent_steps_trained: 344

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v0_80d83_00000,TERMINATED,172.28.0.2:3156,86,730.413,344000,196.11,200,111,196.11


2022-04-24 09:23:49,677	INFO tune.py:702 -- Total run time: 750.20 seconds (749.72 seconds for the tuning loop).


In [ ]:

parameter_search_config = {
    "env": "CartPole-v0",
    "framework": "torch",

    # Hyperparameter tuning
    "model": {
      "fcnet_hiddens": ray.tune.grid_search([[32], [64]]),
      "fcnet_activation": ray.tune.grid_search(["linear", "relu"]),
      "dueling": ray.tune.grid_search([True, False]),
      "double_q": ray.tune.grid_search([True, False])
    },
    "lr": ray.tune.uniform(1e-7, 1e-2)
}

# To explicitly stop or restart Ray, use the shutdown API.
ray.shutdown()

ray.init(
  num_cpus=12,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)

parameter_search_analysis = ray.tune.run(
  "PPO",
  config=parameter_search_config,
  stop=stop,
  num_samples=5,
  metric="timesteps_total",
  mode="min",
)

print(
  "Best hyperparameters found:",
  parameter_search_analysis.best_config,
)

2022-04-24 09:23:56,238	INFO trial_runner.py:803 -- starting PPO_CartPole-v0_43c3f_00000


# Model building
 

In [ ]:
try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

In [ ]:
env.render()

In [ ]:
def plot_environment(env, figsize=(5,4)):
    plt.figure(figsize=figsize)
    img = env.render(mode="rgb_array")
    plt.imshow(img)
    plt.axis("off")
    return img

In [ ]:
plot_environment(env)
plt.show()

In [ ]:
def play_one_step(env, state, epsilon):
    action = epsilon_greedy_policy(state, epsilon)
    next_state, reward, done, info = env.step(action)
    replay_memory.append((state, action, reward, next_state, done))
    return next_state, reward, done, info

In [ ]:
def render_policy_net(model, n_max_steps=200, seed=42):
    frames = []
    env = gym.make("CartPole-v1")
    env.seed(seed)
    np.random.seed(seed)
    obs = env.reset()
    for step in range(n_max_steps):
        frames.append(env.render(mode="rgb_array"))
        left_proba = model.predict(obs.reshape(1, -1))
        action = int(np.random.rand() > left_proba)
        obs, reward, done, info = env.step(action)
        if done:
            break
    env.close()
    return frames

In [ ]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)
    else:
        Q_values = model.predict(state[np.newaxis])
        return np.argmax(Q_values[0])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

K = keras.backend
input_states = keras.layers.Input(shape=[4])
hidden1 = keras.layers.Dense(32, activation="elu")(input_states)
hidden2 = keras.layers.Dense(32, activation="elu")(hidden1)
state_values = keras.layers.Dense(1)(hidden2)
raw_advantages = keras.layers.Dense(n_outputs)(hidden2)
advantages = raw_advantages - K.max(raw_advantages, axis=1, keepdims=True)
Q_values = state_values + advantages
model = keras.models.Model(inputs=[input_states], outputs=[Q_values])

target = keras.models.clone_model(model)
target.set_weights(model.get_weights())

In [ ]:
batch_size = 32
discount_rate = 0.95
optimizer = keras.optimizers.Adam(learning_rate=7.5e-3)
loss_fn = keras.losses.Huber()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones = experiences
    next_Q_values = model.predict(next_states)
    best_next_actions = np.argmax(next_Q_values, axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    next_best_Q_values = (target.predict(next_states) * next_mask).sum(axis=1)
    target_Q_values = (rewards + 
                       (1 - dones) * discount_rate * next_best_Q_values)
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
replay_memory = deque(maxlen=2000)

In [ ]:
env.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

rewards = []
best_score = 0

for episode in range(600):
    obs = env.reset()    
    for step in range(200):
        epsilon = max(1 - episode / 500, 0.01)
        obs, reward, done, info = play_one_step(env, obs, epsilon)
        if done:
            break
    rewards.append(step)
    if step >= best_score:
        best_weights = model.get_weights()
        best_score = step
    print("\rEpisode: {}, Steps: {}, eps: {:.3f}".format(episode, step + 1, epsilon), end="")
    if episode >= 50:
        training_step(batch_size)
        if episode % 50 == 0:
            target.set_weights(model.get_weights())

model.set_weights(best_weights)

In [ ]:
plt.plot(rewards)
plt.xlabel("Episode")
plt.ylabel("Sum of rewards")
plt.show()